In [1]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
import os
import json
import torch
from torchvision.ops import box_convert
import matplotlib.pyplot as plt

model = load_model("GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "GroundingDINO/weights/groundingdino_swint_ogc.pth")
# IMAGE_PATH = "/home/youxiang/Desktop/FYP/WSSS_method/VOCtrainval_11-May-2012/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg"
# TEXT_PROMPT = "aeroplane. person."
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

curr_dir = os.getcwd()
voc_root = os.path.join(curr_dir, "VOCtrainval_11-May-2012", "VOCdevkit", "VOC2012")

if not os.path.isdir(voc_root):
    raise RuntimeError("VOC Directory is wrong")

splits_dir = os.path.join(voc_root, "ImageSets", "Segmentation")
splits_f = os.path.join(splits_dir, "{}.txt".format("train"))

with open(os.path.join(splits_f)) as f:
    file_names = [x.strip() for x in f.readlines()]

img_dir = os.path.join(voc_root, "JPEGImages")



/home/youxiang/anaconda3/envs/voc_wsss/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/youxiang/anaconda3/envs/voc_wsss/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392026823/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [2]:
for x in file_names:
  point_file = "voc_points/{}_points.json".format(x)
  image_loc = os.path.join(img_dir, "{}.jpg".format(x))
  image_source, image = load_image(image_loc)

  with open(point_file, "r") as input_file:
    whole_input = json.load(input_file)

  if whole_input == [] or len(whole_input) == 0:
    print("input is empty")
    continue
  
  categories = []
  # points = []
  for object in whole_input:
    category = object["category"]
    # sampled_points = object["sampled_points"]
    categories.append(category)
    # points.append(sampled_points)

  categories = list(set(categories))
  cat_str = '. '.join(categories) + "."

  boxes, logits, phrases = predict(
    model=model,
    image=image,
    caption=cat_str,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD
  )

  h, w, _ = image_source.shape
  boxes = boxes * torch.tensor([w, h, w, h])
  xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()
  xyxy = torch.tensor(xyxy, dtype=torch.int32)
  xyxy = xyxy.numpy().astype(int)
  # sort by area
  areas = []
  for box in xyxy:
    x1, y1, x2, y2 = box
    area = (x2 - x1) * (y2 - y1)
    areas.append(area)

  k = len(areas)
  # print(areas.index(max(areas)))
  sorted_boxes = []
  for idx in range(k):
    j = areas.index(min(areas))
    sorted_boxes.append(xyxy[j])
    areas[j] = float('inf')

  # print(sorted_boxes)
  #It's in the form of Top-left, bottom-right
  # print(xyxy)
  for object2 in whole_input:
    # print(object2['category'])
    points = object2["sampled_points"]
    for i in range(len(sorted_boxes)):
      flag = 0  
      box = sorted_boxes[i]
      # print(box)
      x1, y1, x2, y2 = box
      assert x2 > x1 and y2 > y1
      for point in points:
        x, y = point
        if x >= x1 and x <= x2 and y >= y1 and y <= y2:
          flag+= 1
      if flag >= 2:
        box = box.tolist()
        # print(box)
        object2.update({"box": box})
        break

  with open(point_file, "w") as output_file:
    json.dump(whole_input, output_file)



# boxes, logits, phrases = predict(
#     model=model,
#     image=image,
#     caption=TEXT_PROMPT,
#     box_threshold=BOX_TRESHOLD,
#     text_threshold=TEXT_TRESHOLD,
# )

# annotated_frame = annotate(
#     image_source=image_source, boxes=boxes, logits=logits, phrases=phrases
# )
# plt.imshow("annotated_image.jpg", annotated_frame)

/home/youxiang/anaconda3/envs/voc_wsss/lib/python3.11/site-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/youxiang/anaconda3/envs/voc_wsss/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/youxiang/anaconda3/envs/voc_wsss/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
